In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Define la zona horaria de Perú
peru_timezone = pytz.timezone('America/Lima')

# Define la fecha de inicio (2014-01-01)
start_date = datetime(2014, 1, 1)

# Número de días para recopilar datos (0 para recopilar solo para la fecha inicial)
num_days = 40

# Si num_days es mayor que 0, calcula la fecha de finalización
if num_days > 0:
    end_date = start_date + timedelta(days=num_days - 1)
else:
    end_date = start_date

# Lista para almacenar los DataFrames de cada página
dfs = []

# Mientras la fecha de inicio sea anterior o igual a la fecha de finalización
while start_date <= end_date:
    # Formatea la fecha en el formato "yyyy-mm-dd"
    formatted_date = start_date.strftime("%Y-%m-%d")

    # Construye la URL con la fecha actual
    cnn_url = f"https://elcomercio.pe/archivo/todas/{formatted_date}/"

    # Realiza la solicitud HTTP
    html = requests.get(cnn_url)
    bsobj = BeautifulSoup(html.content, 'lxml')

    # Inicializa las listas para almacenar los datos
    headlines = []
    redaccion = []
    dates = []
    images = []
    links = []
    comments = []

    # Tu código de extracción de datos aquí
    for link in bsobj.findAll("h2"):
        headlines.append(link.text)

    for news in bsobj.find_all('a', {'class': 'story-item__author block uppercase mt-10 font-thin text-xs text-gray-200'}):
        author_name = news.text.strip()

        # Reemplazar espacios en blanco con "No hay" si el nombre está vacío
        if not author_name:
            author_name = "No hay"

        # Agregar el nombre a la lista
        redaccion.append(author_name)

    for news in bsobj.find_all('p', {'class': 'story-item__date font-thin ml-5 text-xs text-gray-300 md:mt-5 md:ml-0'}):
        dates.append(news.text.strip())

    for news in bsobj.find_all('img', {'class': 'story-item__img object-cover object-center w-full h-full'}):
        img_url = news.get('src')
        if img_url:
            images.append(img_url)

    for index, news in enumerate(bsobj.find_all('a', {'class': 'story-item__title block overflow-hidden primary-font line-h-xs mt-10'})):
        news_link = news.get('href')
        if news_link:
            full_link = "https://elcomercio.pe" + news_link
            links.append(full_link)
            #print(full_link)  # Imprimir el enlace
        else:
            link_text = "No tiene link"
            links.append(link_text)

            # Imprimir el mensaje cuando no se encuentra un enlace
            #print(f"Iteración {index}: {link_text}")

    for link in links:
        if link == "No tiene link":
            comment_text = "No hay comentario disponible"
        else:
            news_html = requests.get(link)
            news_bsobj = BeautifulSoup(news_html.content, 'lxml')

            # Buscar el primer tipo de elemento que contiene el comentario
            comment_element = news_bsobj.find('h2', {'class': 'sht__summary'})

            # Si no se encontró el primer tipo de elemento, buscar el segundo tipo
            if not comment_element:
                comment_element = news_bsobj.find('h2', {'class': 'story-header__news-summary pr-20 pl-20 mb-20 secondary-font line-h-sm text-gray-300 text-xl font-normal'})

            if comment_element:
                comment_text = comment_element.text.strip()
            else:
                comment_text = "No hay comentario disponible"

        comments.append(comment_text)
        #print(comment_text)

    print(len(headlines))
    print(len(redaccion))
    print(len(dates))
    print(len(images))
    print(len(links))
    print(len(comments))

    # Crear un DataFrame con los datos recopilados
    df = pd.DataFrame({
        'Headline': headlines,
        'Redaccion': redaccion,
        'Date': dates,
        'Image URL': images,
        'Link': links,
        'Comentarios': comments
    })

    # Agregar el DataFrame a la lista de DataFrames
    dfs.append(df)

    # Incrementa la fecha de inicio para la siguiente iteración
    start_date += timedelta(days=1)

# Concatena todos los DataFrames en uno solo
final_df = pd.concat(dfs, ignore_index=True)
pd.set_option('display.max_rows', None)

# Imprime el DataFrame final
print(final_df)

# Guarda el DataFrame final en un archivo CSV
final_df.to_csv('elcomercio_data3.csv', index=False)
final_df.to_excel('elcomercio_data3.xlsx', index=False)

